In [1]:
import pandas as pd
from bert import Ner
from tqdm import tqdm

In [2]:
tqdm.pandas()

C:\Users\gabriel.oliveira.luz\AppData\Local\Continuum\anaconda3\envs\projetofinal2\lib\site-packages\tqdm\std.py:725: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [2]:
!pip list

Package              Version            
-------------------- -------------------
argon2-cffi          20.1.0             
async-generator      1.10               
attrs                20.3.0             
backcall             0.2.0              
beautifulsoup4       4.9.3              
bleach               3.2.1              
blis                 0.4.1              
boto3                1.13.8             
botocore             1.16.8             
catalogue            1.0.0              
certifi              2020.6.20          
cffi                 1.14.0             
chardet              3.0.4              
click                7.1.2              
colorama             0.4.4              
cycler               0.10.0             
cymem                2.0.3              
decorator            4.4.2              
defusedxml           0.6.0              
docutils             0.15.2             
en-core-web-lg       2.3.0              
en-core-web-sm       2.3.0              
entrypoints     

In [3]:
df = pd.read_csv('data_test_ber/ner_dataset.csv', encoding="ISO-8859-1")  
df.head(5)

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,NaN,of,IN,O
2,NaN,demonstrators,NNS,O
3,NaN,have,VBP,O
4,NaN,marched,VBN,O


In [4]:
df['sentence2'] = df.apply(lambda x: 0 if pd.isna(x['Sentence #']) else 1, axis=1)
df.head()

,Sentence #,Word,POS,Tag,sentence2
0,Sentence: 1,Thousands,NNS,O,1
1,NaN,of,IN,O,0
2,NaN,demonstrators,NNS,O,0
3,NaN,have,VBP,O,0
4,NaN,marched,VBN,O,0


In [5]:
df['sentence2'] = df['sentence2'].cumsum()

In [8]:
df.head()

,Sentence #,Word,POS,Tag,sentence2
0,Sentence: 1,Thousands,NNS,O,1
1,NaN,of,IN,O,1
2,NaN,demonstrators,NNS,O,1
3,NaN,have,VBP,O,1
4,NaN,marched,VBN,O,1


In [9]:
de_para = {
    'B-gpe': 'B-MISC',
    'I-gpe': 'I-MISC',
    'O-gpe': 'O-MISC',
    'B-tim': 'B-MISC',
    'I-tim': 'I-MISC',
    'O-tim': 'O-MISC',
    'B-art': 'B-MISC',
    'I-art': 'I-MISC',
    'O-art': 'O-MISC',
    'B-eve': 'B-MISC',
    'I-eve': 'I-MISC',
    'O-eve': 'O-MISC',
    'B-nat': 'B-MISC',
    'I-nat': 'I-MISC',
    'O-nat': 'O-MISC',
    'B-geo':'B-LOC',
    'I-geo':'I-LOC',
    'O-geo':'O-LOC',
    'B-org':'B-ORG',
    'I-org':'I-ORG',
    '0-org':'0-ORG',
    'B-per':'B-PER',
    'I-per':'I-PER',
    'O-per':'O-PER'
}

In [10]:
df['Tag'] = df['Tag'].replace(de_para)

In [12]:
df.head(10)

,Sentence #,Word,POS,Tag,sentence2
0,Sentence: 1,Thousands,NNS,O,1
1,NaN,of,IN,O,1
2,NaN,demonstrators,NNS,O,1
3,NaN,have,VBP,O,1
4,NaN,marched,VBN,O,1
5,NaN,through,IN,O,1
6,NaN,London,NNP,B-LOC,1
7,NaN,to,TO,O,1
8,NaN,protest,VB,O,1
9,NaN,the,DT,O,1


In [13]:
model = Ner("out_base/")

In [14]:
out = model.predict('I live in Brazil')
print(out)

[{'word': 'I', 'tag': 'O', 'confidence': 0.9991158843040466}, {'word': 'live', 'tag': 'O', 'confidence': 0.9997305274009705}, {'word': 'in', 'tag': 'O', 'confidence': 0.9997486472129822}, {'word': 'Brazil', 'tag': 'B-LOC', 'confidence': 0.9995967745780945}]


In [15]:
df_to_model = df.groupby(['sentence2'])['Word'].apply(' '.join).reset_index()

In [18]:
df_to_model.head()

,sentence2,Word
0,1,Thousands of demonstrators have marched throug...
1,2,Families of soldiers killed in the conflict jo...
2,3,They marched from the Houses of Parliament to ...
3,4,"Police put the number of marchers at 10,000 wh..."
4,5,The protest comes on the eve of the annual con...


In [19]:
df_to_model = df_to_model.iloc[:200]

In [20]:
df_to_model.count

<bound method DataFrame.count of      sentence2                                               Word
0            1  Thousands of demonstrators have marched throug...
1            2  Families of soldiers killed in the conflict jo...
2            3  They marched from the Houses of Parliament to ...
3            4  Police put the number of marchers at 10,000 wh...
4            5  The protest comes on the eve of the annual con...
..         ...                                                ...
195        196  The United Nations says December 's Indian Oce...
196        197  The U.N. 's Food and Agricultural Organization...
197        198  It said the loss was significant in a region w...
198        199  The FAO 's estimate includes damage to fishing...
199        200  The agency says it has sent experts to help re...

[200 rows x 2 columns]>

In [21]:
df_to_model['result'] = df_to_model.progress_apply(lambda x: model.predict(x['Word']), axis=1)

100%|██████████| 200/200 [01:24<00:00,  2.38it/s]


In [22]:
df_to_model.head(10)

,sentence2,Word,result
0,1,Thousands of demonstrators have marched throug...,"[{'word': 'Thousands', 'tag': 'O', 'confidence..."
1,2,Families of soldiers killed in the conflict jo...,"[{'word': 'Families', 'tag': 'O', 'confidence'..."
2,3,They marched from the Houses of Parliament to ...,"[{'word': 'They', 'tag': 'O', 'confidence': 0...."
3,4,"Police put the number of marchers at 10,000 wh...","[{'word': 'Police', 'tag': 'O', 'confidence': ..."
4,5,The protest comes on the eve of the annual con...,"[{'word': 'The', 'tag': 'O', 'confidence': 0.9..."
5,6,The party is divided over Britain 's participa...,"[{'word': 'The', 'tag': 'O', 'confidence': 0.9..."
6,7,The London march came ahead of anti-war protes...,"[{'word': 'The', 'tag': 'O', 'confidence': 0.9..."
7,8,The International Atomic Energy Agency is to h...,"[{'word': 'The', 'tag': 'O', 'confidence': 0.9..."
8,9,Iran this week restarted parts of the conversi...,"[{'word': 'Iran', 'tag': 'B-LOC', 'confidence'..."
9,10,Iranian officials say they expect to get acces...,"[{'word': 'Iranian', 'tag': 'B-MISC', 'confide..."


In [24]:
df_ent = df[df['Tag'] != 'O']
df_ent.head()

,Sentence #,Word,POS,Tag,sentence2
6,NaN,London,NNP,B-LOC,1
12,NaN,Iraq,NNP,B-LOC,1
18,NaN,British,JJ,B-MISC,1
42,NaN,Bush,NNP,B-PER,2
65,NaN,Hyde,NNP,B-LOC,3


In [25]:
df_ent['dict'] = df_ent.apply(lambda x: {x['Word']: x['Tag']}, axis=1)
df_ent.head()

C:\Users\gabriel.oliveira.luz\AppData\Local\Continuum\anaconda3\envs\projetofinal2\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Sentence #,Word,POS,Tag,sentence2,dict
6,NaN,London,NNP,B-LOC,1,{'London': 'B-LOC'}
12,NaN,Iraq,NNP,B-LOC,1,{'Iraq': 'B-LOC'}
18,NaN,British,JJ,B-MISC,1,{'British': 'B-MISC'}
42,NaN,Bush,NNP,B-PER,2,{'Bush': 'B-PER'}
65,NaN,Hyde,NNP,B-LOC,3,{'Hyde': 'B-LOC'}


In [30]:
#df_ent.groupby(['sentence2'])['dict'].apply(lambda x: list(x))#.reset_index()
df_list = df_ent.groupby(['sentence2'])['dict'].apply(lambda x: list(x))#.reset_index()
df_list.head(10)

sentence2
1     [{'London': 'B-LOC'}, {'Iraq': 'B-LOC'}, {'Bri...
2                                   [{'Bush': 'B-PER'}]
3                [{'Hyde': 'B-LOC'}, {'Park': 'I-LOC'}]
5     [{'Britain': 'B-LOC'}, {'Labor': 'B-ORG'}, {'P...
6     [{'Britain': 'B-MISC'}, {'Iraq': 'B-LOC'}, {'B...
7     [{'London': 'B-LOC'}, {'Rome': 'B-LOC'}, {'Par...
8     [{'International': 'B-ORG'}, {'Atomic': 'I-ORG...
9            [{'Iran': 'B-MISC'}, {'Isfahan': 'B-LOC'}]
10    [{'Iranian': 'B-MISC'}, {'Wednesday': 'B-MISC'...
12    [{'European': 'B-ORG'}, {'Union': 'I-ORG'}, {'...
Name: dict, dtype: object

In [32]:
#df_final = pd.merge(df_to_model, df_list.loc[:,['sentence2','list']], how='left', on='sentence2') 
df_final = pd.merge(df_to_model, df_list, how='left', on='sentence2') 
df_final.head(10)

,sentence2,Word,result,dict
0,1,Thousands of demonstrators have marched throug...,"[{'word': 'Thousands', 'tag': 'O', 'confidence...","[{'London': 'B-LOC'}, {'Iraq': 'B-LOC'}, {'Bri..."
1,2,Families of soldiers killed in the conflict jo...,"[{'word': 'Families', 'tag': 'O', 'confidence'...",[{'Bush': 'B-PER'}]
2,3,They marched from the Houses of Parliament to ...,"[{'word': 'They', 'tag': 'O', 'confidence': 0....","[{'Hyde': 'B-LOC'}, {'Park': 'I-LOC'}]"
3,4,"Police put the number of marchers at 10,000 wh...","[{'word': 'Police', 'tag': 'O', 'confidence': ...",NaN
4,5,The protest comes on the eve of the annual con...,"[{'word': 'The', 'tag': 'O', 'confidence': 0.9...","[{'Britain': 'B-LOC'}, {'Labor': 'B-ORG'}, {'P..."
5,6,The party is divided over Britain 's participa...,"[{'word': 'The', 'tag': 'O', 'confidence': 0.9...","[{'Britain': 'B-MISC'}, {'Iraq': 'B-LOC'}, {'B..."
6,7,The London march came ahead of anti-war protes...,"[{'word': 'The', 'tag': 'O', 'confidence': 0.9...","[{'London': 'B-LOC'}, {'Rome': 'B-LOC'}, {'Par..."
7,8,The International Atomic Energy Agency is to h...,"[{'word': 'The', 'tag': 'O', 'confidence': 0.9...","[{'International': 'B-ORG'}, {'Atomic': 'I-ORG..."
8,9,Iran this week restarted parts of the conversi...,"[{'word': 'Iran', 'tag': 'B-LOC', 'confidence'...","[{'Iran': 'B-MISC'}, {'Isfahan': 'B-LOC'}]"
9,10,Iranian officials say they expect to get acces...,"[{'word': 'Iranian', 'tag': 'B-MISC', 'confide...","[{'Iranian': 'B-MISC'}, {'Wednesday': 'B-MISC'..."


In [42]:
df_final.iloc[0,2:4]

result    [{'word': 'Thousands', 'tag': 'O', 'confidence...
dict      [{'London': 'B-LOC'}, {'Iraq': 'B-LOC'}, {'Bri...
Name: 0, dtype: object